In [13]:
import pandas as pd
import numpy as np
from datetime import datetime

from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest,f_classif
from sklearn.metrics import balanced_accuracy_score,precision_score,recall_score,f1_score,roc_auc_score

In [14]:
df = pd.read_csv('../data/crawler/unified-events-statistics.csv')
def qtd_eventos(integer):
    return 45 - integer

qtd_eventos = list(map(qtd_eventos, df.isnull().sum(axis=1).tolist()))
df['qtd_eventos'] = qtd_eventos
df = df[df.qtd_eventos > 20]
df = df.replace(['BLUE: first_blood',
    'BLUE: dragon',
    'BLUE: herald',
    'BLUE: first_tower_top',
    'BLUE: first_tower_mid',
    'BLUE: first_tower_bot',
    'BLUE: second_tower_top',
    'BLUE: second_tower_mid',
    'BLUE: second_tower_bot',
    'BLUE: third_tower_top',
    'BLUE: third_tower_mid',
    'BLUE: third_tower_bot',
    'BLUE: inhibitor_top',
    'BLUE: inhibitor_mid',
    'BLUE: inhibitor_bot',
    'BLUE: baron',
    'BLUE: elder_dragon',
    'BLUE: nexus_tower',
    'BLUE: nexus',
    'RED: first_blood',
    'RED: dragon',
    'RED: herald',
    'RED: first_tower_top',
    'RED: first_tower_mid',
    'RED: first_tower_bot',
    'RED: second_tower_top',
    'RED: second_tower_mid',
    'RED: second_tower_bot',
    'RED: third_tower_top',
    'RED: third_tower_mid',
    'RED: third_tower_bot',
    'RED: inhibitor_top',
    'RED: inhibitor_mid',
    'RED: inhibitor_bot',
    'RED: baron',
    'RED: elder_dragon',
    'RED: nexus_tower',
    'RED: nexus'], range(38))
df

,golId,game,event1,event2,event3,event4,event5,event6,event7,event8,...,redMidWR,redMidKDA,redAdcGP,redAdcWR,redAdcKDA,redSupportGP,redSupportWR,redSupportKDA,result,qtd_eventos
0,36864,ESPORTSTMNT02_2556988,19,1,2,1,3,24,2,23,...,0.71,4.2,1,0.00,2.5,1,1.00,7.0,1,28
2,36866,ESPORTSTMNT02_2557390,19,2,20,24,2,4,20,3,...,0.80,4.8,1,0.00,0.8,5,0.60,3.2,0,31
3,36867,ESPORTSTMNT02_2557426,20,0,21,20,21,5,23,1,...,0.67,3.9,34,0.74,7.3,2,0.00,2.0,0,25
4,36868,ESPORTSTMNT02_2557463,19,1,2,1,2,4,20,24,...,0.40,2.4,2,1.00,24.0,1,1.00,6.0,0,23
6,36888,ESPORTSTMNT02_2557008,1,21,0,3,20,5,8,2,...,0.00,2.5,2,0.50,5.7,0,0.00,0.0,1,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4104,44378,ESPORTSTMNT02_3057923,21,19,1,24,3,21,23,1,...,0.47,2.9,26,0.81,7.6,14,0.57,3.6,1,24
4105,44379,ESPORTSTMNT02_3047802,0,1,2,1,24,3,27,4,...,0.53,3.2,0,0.00,0.0,4,0.75,1.9,1,25
4107,44381,ESPORTSTMNT02_3047847,21,19,20,24,2,1,22,5,...,0.54,5.7,21,0.38,4.2,3,1.00,7.0,1,24
4110,44383,ESPORTSTMNT02_3047878,0,20,21,1,24,21,22,23,...,0.62,6.1,13,0.69,4.8,11,0.82,10.4,0,27


In [15]:
y = df['result'].copy()
X = df.drop(['golId','result','game','event21','event22','event23','event24','event25','event26','event27','event28','event29','event30','event31','event32','event33','event34','event35','event36','event37','event38','event39','event40','event41','event42','event43','event44','event45'],axis=1)
X

,event1,event2,event3,event4,event5,event6,event7,event8,event9,event10,...,redMidGP,redMidWR,redMidKDA,redAdcGP,redAdcWR,redAdcKDA,redSupportGP,redSupportWR,redSupportKDA,qtd_eventos
0,19,1,2,1,3,24,2,23,6,1,...,14,0.71,4.2,1,0.00,2.5,1,1.00,7.0,28
2,19,2,20,24,2,4,20,3,6,1,...,5,0.80,4.8,1,0.00,0.8,5,0.60,3.2,31
3,20,0,21,20,21,5,23,1,20,26,...,15,0.67,3.9,34,0.74,7.3,2,0.00,2.0,25
4,19,1,2,1,2,4,20,24,5,22,...,5,0.40,2.4,2,1.00,24.0,1,1.00,6.0,23
6,1,21,0,3,20,5,8,2,24,4,...,1,0.00,2.5,2,0.50,5.7,0,0.00,0.0,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4104,21,19,1,24,3,21,23,1,15,1,...,19,0.47,2.9,26,0.81,7.6,14,0.57,3.6,24
4105,0,1,2,1,24,3,27,4,6,2,...,40,0.53,3.2,0,0.00,0.0,4,0.75,1.9,25
4107,21,19,20,24,2,1,22,5,3,4,...,13,0.54,5.7,21,0.38,4.2,3,1.00,7.0,24
4110,0,20,21,1,24,21,22,23,20,5,...,21,0.62,6.1,13,0.69,4.8,11,0.82,10.4,27


In [16]:
def preprocess_input(X,y):
    X = X.copy()
    X_train,X_test,y_train,y_test = train_test_split(X,y,train_size=0.8,random_state=42, stratify=y)
    scaler = StandardScaler()   
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    return X_train,X_test,y_train,y_test
X_train,X_test,y_train,y_test = preprocess_input(X,y)

In [17]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC, SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier

models = {
    'Logistic Regression': LogisticRegression(random_state=42,max_iter=10000),
    'Support Vector Machine (Linear Kernel)': LinearSVC(random_state=42,max_iter=10000),
    'Support Vector Machine (RBF Kernel)': SVC(random_state=42,C=1,gamma=0.01,kernel='sigmoid',max_iter=50000),
    'Decission Tree': DecisionTreeClassifier(random_state=42),
    'Adaboost': AdaBoostClassifier(random_state=42),
    'Random Forest': RandomForestClassifier(random_state=42),
    'Gradient Boosting Classifier': GradientBoostingClassifier(random_state=42)
}

GBCmodel = GradientBoostingClassifier()
GBCmodel.fit(X_train,y_train)


for name, model in models.items():
  model.fit(X_train,y_train)
  print(name + ' trained')

Logistic Regression trained
Support Vector Machine (Linear Kernel) trained


c:\Users\linco\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\svm\_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Support Vector Machine (RBF Kernel) trained
Decission Tree trained
Adaboost trained
Random Forest trained
Gradient Boosting Classifier trained


In [18]:
scores_list = []

for name,model in models.items():    
    scores_list.append({
    'Model': name,
    'Balanced Accuracy': balanced_accuracy_score(y_test,model.predict(X_test)),
    'Precision':  precision_score(y_test,model.predict(X_test)),
    'Recall': recall_score(y_test,model.predict(X_test)),
    'F1-Score': f1_score(y_test,model.predict(X_test)),
    'ROC/AUC Score': roc_auc_score(y_test,model.predict(X_test))
    })
scores = pd.DataFrame(scores_list)

In [19]:
scores

,Model,Balanced Accuracy,Precision,Recall,F1-Score,ROC/AUC Score
0,Logistic Regression,0.903783,0.904624,0.904624,0.904624,0.903783
1,Support Vector Machine (Linear Kernel),0.900842,0.899425,0.904624,0.902017,0.900842
2,Support Vector Machine (RBF Kernel),0.902465,0.916418,0.887283,0.901615,0.902465
3,Decission Tree,0.877618,0.885294,0.869942,0.877551,0.877618
4,Adaboost,0.896506,0.898551,0.895954,0.897250,0.896506
5,Random Forest,0.909665,0.915205,0.904624,0.909884,0.909665
6,Gradient Boosting Classifier,0.915598,0.931138,0.898844,0.914706,0.915598


In [20]:
validation_games = [
'ESPORTSTMNT02_3041846',
'ESPORTSTMNT02_3041862',
'ESPORTSTMNT02_3041903',
'ESPORTSTMNT02_3041937',
'ESPORTSTMNT02_3041983',
'ESPORTSTMNT02_3041990',
'ESPORTSTMNT02_3042003',
'ESPORTSTMNT02_3042025',
'ESPORTSTMNT02_3042187',
'ESPORTSTMNT02_3042251',
'ESPORTSTMNT02_3052761',
'ESPORTSTMNT02_3052797',
'ESPORTSTMNT02_3042301',
'ESPORTSTMNT02_3052853',
'ESPORTSTMNT02_3052865',
'ESPORTSTMNT02_3053150',
'ESPORTSTMNT02_3053187',
'ESPORTSTMNT02_3053204',
'ESPORTSTMNT02_3053238',
'ESPORTSTMNT02_3053254',
'ESPORTSTMNT02_3053274',
'ESPORTSTMNT02_3053295',
'ESPORTSTMNT02_3053311',
'ESPORTSTMNT02_3042846',
'ESPORTSTMNT02_3042864',
'ESPORTSTMNT02_3042881',
'ESPORTSTMNT02_3042903',
'ESPORTSTMNT02_3042921',
'ESPORTSTMNT02_3042943',
'ESPORTSTMNT02_3042965',
'ESPORTSTMNT02_3042987',
'ESPORTSTMNT02_3043190',
'ESPORTSTMNT02_3043245',
'ESPORTSTMNT02_3043269',
'ESPORTSTMNT02_3043280',
'ESPORTSTMNT02_3043302',
'ESPORTSTMNT02_3043333',
'ESPORTSTMNT02_3043345',
'ESPORTSTMNT02_3043363',
'ESPORTSTMNT02_3043656',
'ESPORTSTMNT02_3043674',
'ESPORTSTMNT02_3043685',
'ESPORTSTMNT02_3043709',
'ESPORTSTMNT02_3043731',
'ESPORTSTMNT02_3043753',
'ESPORTSTMNT02_3043758',
'ESPORTSTMNT02_3045120',
'ESPORTSTMNT02_3045181',
'ESPORTSTMNT02_3045196',
'ESPORTSTMNT02_3045209',
'ESPORTSTMNT02_3045232',
'ESPORTSTMNT02_3045259',
'ESPORTSTMNT02_3045416',
'ESPORTSTMNT02_3045459',
'ESPORTSTMNT02_3045472',
'ESPORTSTMNT02_3045480',
'ESPORTSTMNT02_3045503',
'ESPORTSTMNT02_3045518',
'ESPORTSTMNT02_3045836',
'ESPORTSTMNT02_3045853',
'ESPORTSTMNT02_3045864',
'ESPORTSTMNT02_3045877',
'ESPORTSTMNT02_3056381',
'ESPORTSTMNT02_3056394',
'ESPORTSTMNT02_3056626',
'ESPORTSTMNT02_3056652',
'ESPORTSTMNT02_3056673',
'ESPORTSTMNT02_3056681',
'ESPORTSTMNT02_3056705',
'ESPORTSTMNT02_3056721',
'ESPORTSTMNT02_3057453',
'ESPORTSTMNT02_3057478',
'ESPORTSTMNT02_3057490',
'ESPORTSTMNT02_3057502',
'ESPORTSTMNT02_3057505',
'ESPORTSTMNT02_3057516',
'ESPORTSTMNT02_3057653',
'ESPORTSTMNT02_3057673',
'ESPORTSTMNT02_3057686',
'ESPORTSTMNT02_3057715',
'ESPORTSTMNT02_3057724',
'ESPORTSTMNT02_3057747',
'ESPORTSTMNT02_3047499',
'ESPORTSTMNT02_3057859',
'ESPORTSTMNT02_3057882',
'ESPORTSTMNT02_3057894',
'ESPORTSTMNT02_3057911',
'ESPORTSTMNT02_3057916',
'ESPORTSTMNT02_3057923',
'ESPORTSTMNT02_3057927',
'ESPORTSTMNT02_3047802',
'ESPORTSTMNT02_3047839',
'ESPORTSTMNT02_3047847',
'ESPORTSTMNT02_3047861',
'ESPORTSTMNT02_3047878',
'ESPORTSTMNT02_3047898',
'ESPORTSTMNT02_3047918'
]

In [21]:
df = pd.read_csv('../data/crawler/unified-events-statistics.csv')
def qtd_eventos(integer):
    return 45 - integer

qtd_eventos = list(map(qtd_eventos, df.isnull().sum(axis=1).tolist()))
df['qtd_eventos'] = qtd_eventos
df = df[df.qtd_eventos > 20]
df = df.replace(['BLUE: first_blood',
    'BLUE: dragon',
    'BLUE: herald',
    'BLUE: first_tower_top',
    'BLUE: first_tower_mid',
    'BLUE: first_tower_bot',
    'BLUE: second_tower_top',
    'BLUE: second_tower_mid',
    'BLUE: second_tower_bot',
    'BLUE: third_tower_top',
    'BLUE: third_tower_mid',
    'BLUE: third_tower_bot',
    'BLUE: inhibitor_top',
    'BLUE: inhibitor_mid',
    'BLUE: inhibitor_bot',
    'BLUE: baron',
    'BLUE: elder_dragon',
    'BLUE: nexus_tower',
    'BLUE: nexus',
    'RED: first_blood',
    'RED: dragon',
    'RED: herald',
    'RED: first_tower_top',
    'RED: first_tower_mid',
    'RED: first_tower_bot',
    'RED: second_tower_top',
    'RED: second_tower_mid',
    'RED: second_tower_bot',
    'RED: third_tower_top',
    'RED: third_tower_mid',
    'RED: third_tower_bot',
    'RED: inhibitor_top',
    'RED: inhibitor_mid',
    'RED: inhibitor_bot',
    'RED: baron',
    'RED: elder_dragon',
    'RED: nexus_tower',
    'RED: nexus'], range(38))

In [22]:
validation_df = df.apply(lambda row: row[df['game'].isin(validation_games)])
y_validation = validation_df['result'].copy()
X_validation = validation_df.drop(['golId','result','game','event21','event22','event23','event24','event25','event26','event27','event28','event29','event30','event31','event32','event33','event34','event35','event36','event37','event38','event39','event40','event41','event42','event43','event44','event45'],axis=1)

In [23]:
val_scores_list = []

for name,model in models.items():    
    val_scores_list.append({
    'Model': name,
    'Balanced Accuracy': balanced_accuracy_score(y_validation,model.predict(X_validation)),
    'Precision':  precision_score(y_validation,model.predict(X_validation)),
    'Recall': recall_score(y_validation,model.predict(X_validation)),
    'F1-Score': f1_score(y_validation,model.predict(X_validation)),
    'ROC/AUC Score': roc_auc_score(y_validation,model.predict(X_validation))
    })
val_scores = pd.DataFrame(val_scores_list)

c:\Users\linco\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\linco\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\linco\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\linco\AppData\Local\Pr

In [24]:
val_scores

,Model,Balanced Accuracy,Precision,Recall,F1-Score,ROC/AUC Score
0,Logistic Regression,0.500000,0.000000,0.000000,0.000000,0.500000
1,Support Vector Machine (Linear Kernel),0.500000,0.000000,0.000000,0.000000,0.500000
2,Support Vector Machine (RBF Kernel),0.500000,0.000000,0.000000,0.000000,0.500000
3,Decission Tree,0.484539,0.589744,0.938776,0.724409,0.484539
4,Adaboost,0.500000,0.000000,0.000000,0.000000,0.500000
5,Random Forest,0.500000,0.000000,0.000000,0.000000,0.500000
6,Gradient Boosting Classifier,0.453927,0.573333,0.877551,0.693548,0.453927
